# Looking at adding_model.py and setup.py

setup.py: One entry point is 'tape-embed = tape.main:run_embed' > training.run_embed: 

```
dataset = task_spec.dataset(data_file, tokenizer=tokenizer) > 
valid_loader = utils.setup_loader(dataset, batch_size, local_rank, n_gpu, 1, num_workers)
```

Ended up at tape.datasets



In [1]:
from tape import datasets
datasets.dataset_factory

<function tape.datasets.dataset_factory(data_file: Union[str, pathlib.Path], *args, **kwargs) -> torch.utils.data.dataset.Dataset>

In [2]:
sprot = datasets.dataset_factory(data_file='../data/uniprot_sprot.fasta')

In [3]:
sprot[1]

{'id': 'sp|Q6GZX3|002L_FRG3G',
 'primary': 'MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQTCASGFCTSQPLCARIKKTQVCGLRYSSKGKDPLVSAEWDSRGAPYVRCTYDADLIDTQAQVDQFVSMFGESPSLAERYCMRGVKNTAGELVSRVSSDADPAGGWCRKWYSAHRGPDQDAALGSFCIKNPGAADCKCINRASDPVYQKVKTLHAYPDQCWYVPCAADVGELKMGTQRDTPTNCPTQVCQIVFNMLDDGSVTMDDVKNTINCDFSKYVPPPPPPKPTPPTPPTPPTPPTPPTPPTPPTPRPVHNRKVMFFVAGAVLVAILISTVRW',
 'protein_length': 320}